Autor: Ricardo Ander-Egg Aguilar

* 🖥: https://ricardoanderegg.com/
* 🐦: https://twitter.com/ricardoanderegg
* 👨🏻‍🎓: https://www.linkedin.com/in/ricardoanderegg/

## ORM, Bases de datos con SQLAlchemy

ORM = Object relational mapper

In [3]:
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy import Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base

In [4]:
# creamos una base de datos SQLite directamente en la memoria RAM del ordenador
engine = create_engine("sqlite:///curso.db")

# para crearla en el disco duro. echo=True hará que SQLAlchemy nos devuelve información del tipo de queries que está ejecutando
# engine = create_engine("sqlite:///curso.db", echo=True)

In [5]:
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

In [6]:
from sqlalchemy import Column, Integer, String

Creamos nuestra tabla.


El método `__repr__` indica dentro de una clase qué queremos que aparezca cuando hagamos `print()`.

In [5]:
class User(Base):
    __tablename__ = "users"

    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    url = Column(String)

    def __repr__(self):
        return "<User(name='%s', fullname='%s', url='%s')>" % (
            self.name,
            self.fullname,
            self.url,
        )

In [6]:
Base.metadata.create_all(engine)

In [7]:
ricardo_user = User(name="ricardo", fullname="Ricardo Ander-Egg", url="ricardoanderegg.com")

In [7]:
from sqlalchemy.orm import sessionmaker

In [9]:
Session = sessionmaker(bind=engine)

In [10]:
session = Session()

In [11]:
session.add(ricardo_user)

Ahora podemos hacer queries sin escribir directamente SQL usando python, objetos, y la relación entre estos objetos.

In [12]:
mi_usuario = session.query(User).filter_by(name="ricardo").first()

Añadimos varios a la vez.

In [13]:
lista_users = [
    User(name="jaume", fullname="jaume goñi", url="twitter.com/jaume"),
    User(name="carles", fullname="carls hernan", url="twitter.com/carles"),
    User(name="david", fullname="davida barnes", url="twitter.com/david"),
    User(name="jose antonio", fullname="jose antonio cuenca", url="twitter.com/jose"),
    User(name="pedro", fullname="pedro roldar", url="twitter.com/pedro"),
    
]

In [14]:
session.add_all(lista_users)

In [15]:
session.commit()

Si nos equivocamos podemos hacer un **`.rollback()`** antes de hacer commit!

In [16]:
usuario_erroneo = User(name="user erroneo", fullname="oriol mitja", url="google.com")

In [17]:
session.add(usuario_erroneo)

Podemos hacer una query 

In [18]:
session.query(User).filter(User.name.in_(["Manuel", "user erroneo"])).all()

[<User(name='user erroneo', fullname='oriol mitja', url='google.com')>]

Hacemos rollback para "deshacer" la acción que habíamos ejecutado con el `.add()`

In [19]:
session.rollback()

Para filtrar tenemos muchas opciones!

In [20]:
usuarios = session.query(User).filter(User.name.in_(["ricardo", "pedro"])).all()

In [21]:
usuarios[0].url = "twitter.com/ricardoanderegg"

In [22]:
u = usuarios[0]

In [23]:
session.commit()

In [24]:
for usuario in session.query(User).order_by(User.id):
    print(usuario.name, usuario.fullname, usuario.url)

ricardo Ricardo Ander-Egg twitter.com/ricardoanderegg
jaume jaume goñi twitter.com/jaume
carles carls hernan twitter.com/carles
david davida barnes twitter.com/david
jose antonio jose antonio cuenca twitter.com/jose
pedro pedro roldar twitter.com/pedro
ricardo Ricardo Ander-Egg ricardoanderegg.com
jaume jaume goñi twitter.com/jaume
carles carls hernan twitter.com/carles
david davida barnes twitter.com/david
jose antonio jose antonio cuenca twitter.com/jose
pedro pedro roldar twitter.com/pedro
jaume jaume serra twitter.com/jaume2
ricardo Ricardo Ander-Egg ricardoanderegg.com
jaume jaume goñi twitter.com/jaume
carles carls hernan twitter.com/carles
david davida barnes twitter.com/david
jose antonio jose antonio cuenca twitter.com/jose
pedro pedro roldar twitter.com/pedro


In [25]:
for nombre, nombre_completo in session.query(User.name, User.fullname):
    print(nombre, nombre_completo)

ricardo Ricardo Ander-Egg
jaume jaume goñi
carles carls hernan
david davida barnes
jose antonio jose antonio cuenca
pedro pedro roldar
ricardo Ricardo Ander-Egg
jaume jaume goñi
carles carls hernan
david davida barnes
jose antonio jose antonio cuenca
pedro pedro roldar
jaume jaume serra
ricardo Ricardo Ander-Egg
jaume jaume goñi
carles carls hernan
david davida barnes
jose antonio jose antonio cuenca
pedro pedro roldar


El equivalente de esto en SQL sería:

```sql
SELECT users.name AS nombre,
        users.fullname AS nombre_completo
FROM users
()
```

In [26]:
for row in session.query(User, User.name).all():
    print(row.User, row.name)

<User(name='ricardo', fullname='Ricardo Ander-Egg', url='twitter.com/ricardoanderegg')> ricardo
<User(name='jaume', fullname='jaume goñi', url='twitter.com/jaume')> jaume
<User(name='carles', fullname='carls hernan', url='twitter.com/carles')> carles
<User(name='david', fullname='davida barnes', url='twitter.com/david')> david
<User(name='jose antonio', fullname='jose antonio cuenca', url='twitter.com/jose')> jose antonio
<User(name='pedro', fullname='pedro roldar', url='twitter.com/pedro')> pedro
<User(name='ricardo', fullname='Ricardo Ander-Egg', url='ricardoanderegg.com')> ricardo
<User(name='jaume', fullname='jaume goñi', url='twitter.com/jaume')> jaume
<User(name='carles', fullname='carls hernan', url='twitter.com/carles')> carles
<User(name='david', fullname='davida barnes', url='twitter.com/david')> david
<User(name='jose antonio', fullname='jose antonio cuenca', url='twitter.com/jose')> jose antonio
<User(name='pedro', fullname='pedro roldar', url='twitter.com/pedro')> pedro
<U

In [27]:
nuevo_user = User(name="jaume", fullname="jaume serra", url="twitter.com/jaume2")

In [28]:
session.add(nuevo_user)

In [29]:
session.commit()

In [30]:
for user in (
    session.query(User)
    .filter(User.name == "jaume")
    .filter(User.fullname == "jaume goñi")
):
    print(user)

<User(name='jaume', fullname='jaume goñi', url='twitter.com/jaume')>
<User(name='jaume', fullname='jaume goñi', url='twitter.com/jaume')>
<User(name='jaume', fullname='jaume goñi', url='twitter.com/jaume')>


In [31]:
nombres_filtro = ["carles", "jaume"]

In [32]:
session.query(User).filter(User.name.in_(nombres_filtro))

In [33]:
import requests
import sqlalchemy

from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String

from sqlalchemy.orm import sessionmaker

In [34]:
class Lenguajes(Base):
    
    __tablename__ = "lenguajes"
    
    id = Column(Integer, primary_key=True)
    name = Column(String, nullable=False)
    year = Column(Integer)

In [35]:
Base.metadata.create_all(engine)

In [36]:
python = Lenguajes(name="python", year=1989)

In [37]:
session.add(python)

In [38]:
session.commit()

In [40]:
session.query(User).all()

[<User(name='ricardo', fullname='Ricardo Ander-Egg', url='twitter.com/ricardoanderegg')>,
 <User(name='jaume', fullname='jaume goñi', url='twitter.com/jaume')>,
 <User(name='carles', fullname='carls hernan', url='twitter.com/carles')>,
 <User(name='david', fullname='davida barnes', url='twitter.com/david')>,
 <User(name='jose antonio', fullname='jose antonio cuenca', url='twitter.com/jose')>,
 <User(name='pedro', fullname='pedro roldar', url='twitter.com/pedro')>,
 <User(name='ricardo', fullname='Ricardo Ander-Egg', url='ricardoanderegg.com')>,
 <User(name='jaume', fullname='jaume goñi', url='twitter.com/jaume')>,
 <User(name='carles', fullname='carls hernan', url='twitter.com/carles')>,
 <User(name='david', fullname='davida barnes', url='twitter.com/david')>,
 <User(name='jose antonio', fullname='jose antonio cuenca', url='twitter.com/jose')>,
 <User(name='pedro', fullname='pedro roldar', url='twitter.com/pedro')>,
 <User(name='jaume', fullname='jaume serra', url='twitter.com/jaume2')

In [ ]:
try:
    session.add(......)
except Exception:
    session.rollback()
    raise

## Ejercicio

1. Obtener todas las cartas de la API.
1. La key `colors` contiene un value que es una lista. Filtrar las cartas que contengan `"Green"` en esa lista.
1. Generar una base de datos con SQLAlchemy para almacenar estas cartas.
1. Las columnas que debemos crear son (entre paréntesis está el nombre de la key que tienen en el diccionario):
    * nombre (`name`)
    * ** multiverse_id (`multiverseid`)  <== queda eliminado a menos que queraís hacer la parte extra
    * url_imagen (`imageUrl`) || tipo (`type`)
    * rareza (`rarity`)
    * **Extra**: en las cartas verdes que NO tienen `multiverseid`, crearlo y darle el valor `0`
    
1. De la lista de cartas filtradas que hemos obtenido en el punto $2$, guardarlas todas en la base de datos.

**Extra 🔥**

* Crear una función que obtenga la información de la carta de la base de datos en base a un `id` o como queráis.
* En las cartas que no tengan la variable `"imageUrl"`, crearla y poner: http://placegoat.com/200/200
* Descargar la imagen de esta carta (o de la cabra 😂) y guardarla en un archivo en el disco.
* Crear un archivo JSON que sea una lista de diccionarios con el siguiente formato:

```python
[
    {"nombre": nombre_carta, "url_imagen": url_imagen},
    {"nombre": nombre_carta, "url_imagen": url_imagen},
    {"nombre": nombre_carta, "url_imagen": url_imagen},
    .
    .
    .
]
```

Por ejemplo:

```python
req_img = requests.get(v[0]["imageUrl"])

with open("imagen.jpg", "wb") as f:
    f.write(req_img.content)
```

In [9]:
import json

with open("cartas.json", "w") as f:
    f.write(json.dumps(cartas))

In [ ]:
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy import Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base

import json

In [2]:
# import requests

# r = requests.get("https://api.magicthegathering.io/v1/cards")

# data = r.json()
# cartas = data["cards"]

In [3]:
with open("cartas.json") as f:
    cartas = json.loads(f.read())

In [4]:
engine = create_engine("sqlite:///cartas_url.db", echo=True)

Base = declarative_base()


class Verde(Base):
    
    __tablename__ = "verdes"

    id = Column(Integer, primary_key=True)
    name = Column(String)
    
    # multiverse_id = Column(Integer)
    url = Column(String)
    
    rarity = Column(String)

    def __repr__(self):
        # url='{self.url}',
        return f"<Verde(name='{self.name}', rarity='{self.rarity}')>"

In [5]:
Base.metadata.create_all(engine)

Session = sessionmaker(bind=engine)

session = Session()

2020-07-09 13:52:30,252 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-07-09 13:52:30,255 INFO sqlalchemy.engine.base.Engine ()
2020-07-09 13:52:30,259 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-07-09 13:52:30,262 INFO sqlalchemy.engine.base.Engine ()
2020-07-09 13:52:30,264 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("verdes")
2020-07-09 13:52:30,266 INFO sqlalchemy.engine.base.Engine ()
2020-07-09 13:52:30,268 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("verdes")
2020-07-09 13:52:30,270 INFO sqlalchemy.engine.base.Engine ()
2020-07-09 13:52:30,272 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE verdes (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	url VARCHAR, 
	rarity VARCHAR, 
	PRIMARY KEY (id)
)


2020-07-09 13:52:30,274 INFO sqlalchemy.engine.base.Engine ()
2020-07-09 13:52:30,280 INFO sqlalchemy.engine.base.Engine COMMIT


In [6]:
verdes = []

for carta in cartas:

    colores = carta["colors"]

    if "Green" in colores:
        try:
            url_imagen = carta["imageUrl"]
        except KeyError:  # no tiene la key ["imageUrl"]
            url_imagen = "http://placegoat.com/200/200"

        nombre = carta["name"]
        rareza = carta["rarity"]

        carta_nueva = Verde(name=nombre, url=url_imagen, rarity=rareza)
        verdes.append(carta_nueva)

In [7]:
verdes

[<Verde(name='Abundance', rarity='Rare')>,
 <Verde(name='Aggressive Urge', rarity='Common')>,
 <Verde(name='Avatar of Might', rarity='Rare')>,
 <Verde(name='Avatar of Might', rarity='Rare')>,
 <Verde(name='Birds of Paradise', rarity='Rare')>,
 <Verde(name='Birds of Paradise', rarity='Rare')>,
 <Verde(name='Blanchwood Armor', rarity='Uncommon')>,
 <Verde(name='Blanchwood Armor', rarity='Uncommon')>,
 <Verde(name='Canopy Spider', rarity='Common')>,
 <Verde(name='Canopy Spider', rarity='Common')>,
 <Verde(name='Civic Wayfinder', rarity='Common')>,
 <Verde(name='Commune with Nature', rarity='Common')>,
 <Verde(name='Craw Wurm', rarity='Common')>,
 <Verde(name='Creeping Mold', rarity='Uncommon')>]

In [8]:
session.add_all(verdes)

In [9]:
session.commit()

2020-07-09 13:54:55,812 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-07-09 13:54:55,814 INFO sqlalchemy.engine.base.Engine INSERT INTO verdes (name, url, rarity) VALUES (?, ?, ?)
2020-07-09 13:54:55,818 INFO sqlalchemy.engine.base.Engine ('Abundance', 'http://gatherer.wizards.com/Handlers/Image.ashx?multiverseid=130483&type=card', 'Rare')
2020-07-09 13:54:55,821 INFO sqlalchemy.engine.base.Engine INSERT INTO verdes (name, url, rarity) VALUES (?, ?, ?)
2020-07-09 13:54:55,822 INFO sqlalchemy.engine.base.Engine ('Aggressive Urge', 'http://gatherer.wizards.com/Handlers/Image.ashx?multiverseid=130525&type=card', 'Common')
2020-07-09 13:54:55,823 INFO sqlalchemy.engine.base.Engine INSERT INTO verdes (name, url, rarity) VALUES (?, ?, ?)
2020-07-09 13:54:55,824 INFO sqlalchemy.engine.base.Engine ('Avatar of Might', 'http://gatherer.wizards.com/Handlers/Image.ashx?multiverseid=135249&type=card', 'Rare')
2020-07-09 13:54:55,825 INFO sqlalchemy.engine.base.Engine INSERT INTO verdes (

In [ ]:
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy import Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base

import json

In [11]:
carta = session.query(Verde).filter(Verde.name == "Craw Wurm").all()

2020-07-09 13:57:22,325 INFO sqlalchemy.engine.base.Engine SELECT verdes.id AS verdes_id, verdes.name AS verdes_name, verdes.url AS verdes_url, verdes.rarity AS verdes_rarity 
FROM verdes 
WHERE verdes.name = ?
2020-07-09 13:57:22,327 INFO sqlalchemy.engine.base.Engine ('Craw Wurm',)


In [13]:
carta[0].url

'http://gatherer.wizards.com/Handlers/Image.ashx?multiverseid=130527&type=card'

In [ ]:
for user in (
    session.query(User)
    .filter(User.name == "daniel")
    .filter(User.fullname == "daniel orejuela")
):
    print(user)

In [ ]:
for nombre, nombre_completo in session.query(User.name, User.fullname):
    print(nombre, nombre_completo)